In [6]:
import pandas as pd
Dataset = pd.read_csv("C:\\Users\\atang\\OneDrive\\Documents\\Data Science MSc\\Individual Research Project\\Database.csv")
Dataset.head()

,Unnamed: 0,Time,Season,Day_of_the_week,DHI,DNI,GHI,Wind_speed,Humidity,Temperature,PV_production,Wind_production,Electric_demand
0,0,2019-01-01-T00:00,1,1,0.0,0.0,0.0,2.88,56.036,1.82,0,2810,22216
1,1,2019-01-01-T00:05,1,1,0.0,0.0,0.0,2.88,56.036,1.82,0,2862,22106
2,2,2019-01-01-T00:10,1,1,0.0,0.0,0.0,2.88,56.194,1.78,0,2916,22130
3,3,2019-01-01-T00:15,1,1,0.0,0.0,0.0,2.88,56.344,1.74,0,2920,22040
4,4,2019-01-01-T00:20,1,1,0.0,0.0,0.0,2.84,56.440,1.72,0,2902,21963


In [7]:
Dataset.shape

(315648, 13)

In [8]:
#Convert the time column to Datetime for time series analysis
Dataset['Time'] = pd.to_datetime(Dataset['Time'])
Dataset.set_index('Time', inplace=True)
Dataset.head()

,Unnamed: 0,Season,Day_of_the_week,DHI,DNI,GHI,Wind_speed,Humidity,Temperature,PV_production,Wind_production,Electric_demand
Time,,,,,,,,,,,,
2019-01-01 00:00:00,0,1,1,0.0,0.0,0.0,2.88,56.036,1.82,0,2810,22216
2019-01-01 00:05:00,1,1,1,0.0,0.0,0.0,2.88,56.036,1.82,0,2862,22106
2019-01-01 00:10:00,2,1,1,0.0,0.0,0.0,2.88,56.194,1.78,0,2916,22130
2019-01-01 00:15:00,3,1,1,0.0,0.0,0.0,2.88,56.344,1.74,0,2920,22040
2019-01-01 00:20:00,4,1,1,0.0,0.0,0.0,2.84,56.440,1.72,0,2902,21963


In [9]:
# Generate a complete date range
full_range = pd.date_range(start=Dataset.index.min(), end=Dataset.index.max(), freq='5min')

# Identify missing dates
missing_dates = full_range.difference(Dataset.index)

# Print the missing dates
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex([], dtype='datetime64[ns]', freq='5min')


In [10]:
# Filter data to keep only the year 2020
Dataset_2020 = Dataset[(Dataset.index.year == 2020)]
Dataset_2020.head()
Dataset_2020.shape

(105408, 12)

In [11]:
Dataset_2020 = Dataset_2020.drop(['Wind_production', 'Electric_demand'], axis = 1)
print(Dataset_2020.columns)

Index(['Unnamed: 0', 'Season', 'Day_of_the_week', 'DHI', 'DNI', 'GHI',
       'Wind_speed', 'Humidity', 'Temperature', 'PV_production'],
      dtype='object')


In [12]:
Dataset_2020['Wind_speed'] = pd.to_numeric(Dataset_2020['Wind_speed'], downcast='float')
Dataset_2020['Season'] = pd.to_numeric(Dataset_2020['Season'], downcast='float')
Dataset_2020['Day_of_the_week'] = pd.to_numeric(Dataset_2020['Day_of_the_week'], downcast='float')
Dataset_2020['DHI'] = pd.to_numeric(Dataset_2020['DHI'], downcast='float')
Dataset_2020['DNI'] = pd.to_numeric(Dataset_2020['DNI'], downcast='float')
Dataset_2020['GHI'] = pd.to_numeric(Dataset_2020['GHI'], downcast='float')
Dataset_2020['Humidity'] = pd.to_numeric(Dataset_2020['Humidity'], downcast='float')
Dataset_2020['Temperature'] = pd.to_numeric(Dataset_2020['Temperature'], downcast='float')
Dataset_2020['PV_production'] = pd.to_numeric(Dataset_2020['PV_production'], downcast='float')

In [13]:
# Replace negative PV_production values with zero
Dataset_2020.loc[Dataset_2020['PV_production'] < 0, 'PV_production'] = 0

In [14]:
# Handle any remaining non-numeric or NA values in PV_production
Dataset_2020 = Dataset_2020.dropna(subset=['PV_production'])
Dataset_2020.shape

(105408, 10)

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

# Specify the target variable and features
y = Dataset_2020['PV_production'].values
X = Dataset_2020.drop(columns=['PV_production']).values

# Normalise the data
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Prepare function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Initialise time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=3)

r2_scores = []
mse_scores = []

# Cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_scaled[train_index], y_scaled[test_index]
    
    # Reshape input to be 3D [samples, timesteps, features]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # Create and fit the LSTM model
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=70, verbose=0)
    
    # Make predictions
    predictions = model.predict(X_test)
    predictions = scaler_y.inverse_transform(predictions)
    y_test = scaler_y.inverse_transform(y_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    
    r2_scores.append(r2)
    mse_scores.append(mse)

# Display the results
results = pd.DataFrame({
    'R2 Score': r2_scores,
    'MSE': mse_scores
})

print(results)


C:\Users\atang\OneDrive\Documents\Data Science MSc\Research Project\Group project\pythonProject3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


824/824 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


C:\Users\atang\OneDrive\Documents\Data Science MSc\Research Project\Group project\pythonProject3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


824/824 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


C:\Users\atang\OneDrive\Documents\Data Science MSc\Research Project\Group project\pythonProject3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


824/824 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step
   R2 Score          MAPE          MSE
0  0.064961  5.116832e+18  19138588.00
1  0.878434  1.186358e+18   2690374.75
2  0.743473  1.769607e+18   3482319.00
